In [1]:
import os
import json
import torch
import jsonlines
import numpy as np

from tqdm import tqdm
from model import BertCRFForTokenClassification
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset

from transformers import (
    AutoConfig,
    AutoTokenizer
)

In [2]:
device = "cuda:0"
pretrained_path = "/tf/FangGexiang/3.SememeV2/pretrained_model/bert-base-chinese"
pretrained_model_path = "/tf/FangGexiang/1.CAILED/ModelSaved/OfficalBaselineV1"

In [3]:
class InputExample(object):
    """
        A single training/test example for token classification.
        one single sequence of tokens is an example in LEVEN task.
    """

    def __init__(self, guid, words, labels):
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

In [4]:
def read_examples_from_file(data_dir):
    input_data = jsonlines.open(data_dir)
    examples = []

    for doc in tqdm(input_data, desc="[1] Reading Examples from file..."):
        words = [c['tokens'] for c in doc['content']]
        labels = [['O']*len(c['tokens']) for c in doc['content']]

        for i in range(0, len(words)):
            examples.append(InputExample(guid="%s-%d" % (doc['id'], i),
                                         words=words[i],
                                         labels=labels[i]))

    return examples

In [5]:
def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=0,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
    model_name=None
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    # my logic in crf_padding requires this check. I create mask for crf by labels==pad_token_label_id to not include it
    # in loss and decoding
    assert pad_token_label_id not in label_map.values()

    features = []
    for ex_index, example in enumerate(tqdm(examples, desc="[2] Converting Examples to Features")):

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            if len(word_tokens) == 0:
                word_tokens = ['<UNK>']
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[:(max_seq_length - special_tokens_count)]
            label_ids = label_ids[:(max_seq_length - special_tokens_count)]

        '''
        The convention in BERT is:
        (a) For sequence pairs:
         tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
         type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        (b) For single sequences:
         tokens:   [CLS] the dog is hairy . [SEP]
         type_ids:   0   0   0   0  0     0   0
        Where "type_ids" are used to indicate whether this is the first
        sequence or the second sequence. The embedding vectors for `type=0` and
        `type=1` were learned during pre-training and are added to the wordpiece
        embedding vector (and position vector). This is not *strictly* necessary
        since the [SEP] token unambiguously separates the sequences, but it makes
        it easier for the model to learn the concept of sequences.
        '''

        tokens += [sep_token]
        label_ids += [pad_token_label_id]       # [label_map["X"]]

        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        if model_name:
            if model_name == 'xlm-roberta-base':
                tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
            elif model_name.startswith('bert'):
                tokenizer = BertTokenizer.from_pretrained(model_name)
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
            elif model_name == 'roberta':
                tokenizer = RobertaTokenizer.from_pretrained(model_name)
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
        else:
            input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids

        else:
            input_ids += ([pad_token] * padding_length)
            input_mask += ([0 if mask_padding_with_zero else 1] * padding_length)
            segment_ids += ([pad_token_segment_id] * padding_length)
            label_ids += ([pad_token_label_id] * padding_length)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        features.append(
            InputFeatures(input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_ids=label_ids))
    return features

In [6]:
def load_and_cache_examples(tokenizer, labels, pad_token_label_id):
    # Load data features from cache or dataset file
    cached_features_file = "/tf/FangGexiang/1.CAILED/Data/Cache/cached_test_stage_1"
    if os.path.exists(cached_features_file):
        features = torch.load(cached_features_file)
    else:
        examples = read_examples_from_file("/tf/FangGexiang/1.CAILED/Data/test_stage1.jsonl")
        features = convert_examples_to_features(
            examples, 
            labels, 
            512,
            tokenizer,
            cls_token_at_end=False,           # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,            # roberta uses an extra separator b/w pairs of sentences
            pad_on_left=False,                # pad on the left for xlnet
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id
        )
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [7]:
def evaluate(model, tokenizer, labels, pad_token_label_id):
    print("正在读取数据...")
    eval_dataset = load_and_cache_examples(tokenizer, labels, pad_token_label_id)

    eval_batch_size = 32
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size)

    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    model.eval()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2]
            }
            best_path = model(pad_token_label_id=pad_token_label_id, **inputs)

        inputs.update({"labels": batch[3]})
        nb_eval_steps += 1
        if preds is None:
            preds = best_path.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, best_path.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    label_map = {i: label for i, label in enumerate(labels)}

    preds_list = [[] for _ in range(out_label_ids.shape[0])]

    for i in range(out_label_ids.shape[0]):
        for j in range(out_label_ids.shape[1]):
            if out_label_ids[i, j] != pad_token_label_id:
                preds_list[i].append(label_map[preds[i][j]])

    return preds_list

In [8]:
labels = eval(open("/tf/FangGexiang/1.CAILED/Data/bio_label.txt", "r").readline())
num_labels = len(labels)

tokenizer = AutoTokenizer.from_pretrained(pretrained_path, use_fast=True)
model_config = AutoConfig.from_pretrained(pretrained_model_path, num_labels=num_labels)

ner_model = BertCRFForTokenClassification.from_pretrained(pretrained_model_path, config=model_config)
ner_model = ner_model.to(device)

In [ ]:
predictions = evaluate(ner_model, tokenizer, labels, -100)

正在读取数据...


Evaluating:  47%|████▋     | 1132/2421 [22:39<26:30,  1.23s/it]

In [ ]:
output_test_predictions_file = "./results.jsonl"
pure_event2id = eval(open("/tf/FangGexiang/1.CAILED/Data/label.txt", "r").readline())
with open(output_test_predictions_file, "w") as writer:
    Cnt = 0
    levenTypes = list(pure_event2id.keys())
    with open("/tf/FangGexiang/1.CAILED/Data/test_stage1.jsonl", "r") as fin:
        lines = fin.readlines()
        for line in tqdm(lines, desc="Writing..."):
            doc = json.loads(line)
            res = {}
            res['id'] = doc['id']
            res['predictions'] = []
            for mention in doc['candidates']:
                if mention['offset'][1] > len(predictions[Cnt + mention['sent_id']]):
                    print(len(doc['content'][mention['sent_id']]['tokens']),
                          len(predictions[Cnt + mention['sent_id']]))
                    res['predictions'].append({"id": mention['id'], "type_id": 0})
                    continue
                is_NA = False if predictions[Cnt + mention['sent_id']][mention['offset'][0]].startswith(
                    "B") else True
                if not is_NA:
                    Type = predictions[Cnt + mention['sent_id']][mention['offset'][0]][2:]
                    for i in range(mention['offset'][0] + 1, mention['offset'][1]):
                        if predictions[Cnt + mention['sent_id']][i][2:] != Type:
                            is_NA = True
                            break
                    if not is_NA:
                        res['predictions'].append({"id": mention['id'], "type_id": levenTypes.index(Type)})
                if is_NA:
                    res['predictions'].append({"id": mention['id'], "type_id": 0})
            writer.write(json.dumps(res) + "\n")
            Cnt += len(doc['content'])